In [6]:
from xml.etree import ElementTree
from xml.dom import minidom
from xml.etree.ElementTree import Element, SubElement, Comment, indent

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, encoding="ISO-8859-1")
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="\t")

In [15]:
import numpy as np
import os

num_hyb = 16
start_hyb = 1
start_valve = 1

valve_ids = np.arange(start_valve, num_hyb + start_valve)
hyb_ids = np.arange(start_hyb, num_hyb + start_hyb)
#reg_names = [f'M{_i}_bit{_i*2-1}-{_i*2}' for _i in hyb_ids]
reg_names = [f'M{_i+1}' for _i in hyb_ids]
print(reg_names)

['M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'M10', 'M11', 'M12', 'M13', 'M14', 'M15', 'M16', 'M17']


In [16]:
source_folder = r'D:\Pu\20230710-4T1cell_50k_zombie'
target_drive = r'\\10.245.74.212\Chromatin_NAS_2\Pu'
imaging_protocol = r'Zscan_750_647_561_488_s30_n500_10Hz'

# MERFISHmax

In [17]:
cmd_seq = Element('command_sequence')

for _vid, _hid, _rname in zip(valve_ids, hyb_ids, reg_names):
    # comments
    comment = Comment(f"Hyb {_hid} for {_rname}")
    cmd_seq.append(comment)
    # TCEP
    tcep = SubElement(cmd_seq, 'valve_protocol')
    #tcep.text = "Flow TCEP"
    tcep.text = "Flow Harsh Wash"
    # wash TCEP with 2xSSC
    tcep_wash = SubElement(cmd_seq, 'valve_protocol')
    tcep_wash.text = "Flow 2xSSC"
    tcep_wash = SubElement(cmd_seq, 'valve_protocol')
    tcep_wash.text = "Flow 2xSSC"
    # flow adaptor
    adt = SubElement(cmd_seq, 'valve_protocol')
    adt.text = f"Hybridize {_vid}"
    # incubate for 15min
    adt = SubElement(cmd_seq, 'valve_protocol')
    adt.text = f"Incubate 15min"
    # wash
    wash = SubElement(cmd_seq, 'valve_protocol')
    wash.text = "Flow Wash Buffer"
    # readouts
    readouts = SubElement(cmd_seq, 'valve_protocol')
    readouts.text = "Flow Imaging Buffer"
    # change directory
    change_dir = SubElement(cmd_seq, 'change_directory')
    change_dir.text = os.path.join(source_folder, f"H{_hid}{_rname.upper()}")
    # wakeup
    wakeup = SubElement(cmd_seq, 'wakeup')
    wakeup.text = "15000"
    
    # hybridization loop
    loop = SubElement(cmd_seq, 'loop', name='Position Loop Zscan', increment="name")
    loop_item = SubElement(loop, 'item', name=imaging_protocol)
    loop_item.text = " "
    # delay time
    delay = SubElement(cmd_seq, 'delay')
    delay.text = "5000"
    # copy folder
    copy_dir = SubElement(cmd_seq, 'copy_directory')
    source_dir = SubElement(copy_dir, 'source_path')
    source_dir.text = change_dir.text#cmd_seq.findall('change_directory')[-1].text
    target_dir = SubElement(copy_dir, 'target_path')
    target_dir.text = os.path.join(target_drive, 
                                   os.path.basename(os.path.dirname(source_dir.text)), 
                                   os.path.basename(source_dir.text))
    del_source = SubElement(copy_dir, 'delete_source')
    del_source.text = "True"
    # empty line
    indent(target_dir)
print( prettify(cmd_seq))

<?xml version="1.0" ?>
<command_sequence>
	<!--Hyb 1 for M2-->
	<valve_protocol>Flow Harsh Wash</valve_protocol>
	<valve_protocol>Flow 2xSSC</valve_protocol>
	<valve_protocol>Flow 2xSSC</valve_protocol>
	<valve_protocol>Hybridize 1</valve_protocol>
	<valve_protocol>Incubate 15min</valve_protocol>
	<valve_protocol>Flow Wash Buffer</valve_protocol>
	<valve_protocol>Flow Imaging Buffer</valve_protocol>
	<change_directory>D:\Pu\20230710-4T1cell_50k_zombie\H1M2</change_directory>
	<wakeup>15000</wakeup>
	<loop name="Position Loop Zscan" increment="name">
		<item name="Zscan_750_647_561_488_s30_n500_10Hz"> </item>
	</loop>
	<delay>5000</delay>
	<copy_directory>
		<source_path>D:\Pu\20230710-4T1cell_50k_zombie\H1M2</source_path>
		<target_path>\\10.245.74.212\Chromatin_NAS_2\Pu\20230710-4T1cell_50k_zombie\H1M2</target_path>
		<delete_source>True</delete_source>
	</copy_directory>
	<!--Hyb 2 for M3-->
	<valve_protocol>Flow Harsh Wash</valve_protocol>
	<valve_protocol>Flow 2xSSC</valve_protocol

In [18]:
# save to a file:
xml_draft_filename = os.path.join(source_folder, f"template_xml_H{start_hyb}-{start_hyb+num_hyb}.txt")
print(xml_draft_filename)
with open(xml_draft_filename, 'w') as _f:
    _f.write(prettify(cmd_seq))

D:\Pu\20230710-4T1cell_50k_zombie\template_xml_H1-17.txt


## RNA without adaptors for IDEX valves

In [6]:
cmd_seq = Element('command_sequence')

for _vid, _hid, _rname in zip(valve_ids, hyb_ids, reg_names):
    # comments
    comment = Comment(f"Hyb {_hid} for {_rname}")
    cmd_seq.append(comment)
    # TCEP
    tcep = SubElement(cmd_seq, 'valve_protocol')
    tcep.text = "Flow TCEP"
    # wash TCEP with 2xSSC
    tcep = SubElement(cmd_seq, 'valve_protocol')
    tcep.text = "Flow 2xSSC"
    # flow adaptor
    adt = SubElement(cmd_seq, 'valve_protocol')
    adt.text = f"Hybridize {_vid}"
    # incubate for 15min
    adt = SubElement(cmd_seq, 'valve_protocol')
    adt.text = f"Incubate 15min"
    # wash
    wash = SubElement(cmd_seq, 'valve_protocol')
    wash.text = "Flow Wash Buffer"
    # readouts
    readouts = SubElement(cmd_seq, 'valve_protocol')
    readouts.text = "Flow Imaging Buffer"
    # change directory
    change_dir = SubElement(cmd_seq, 'change_directory')
    change_dir.text = os.path.join(source_folder, f"H{_hid}{_rname.upper()}")
    # wakeup
    wakeup = SubElement(cmd_seq, 'wakeup')
    wakeup.text = "5000"
    
    # hybridization loop
    loop = SubElement(cmd_seq, 'loop', name='Position Loop Zscan', increment="name")
    loop_item = SubElement(loop, 'item', name=imaging_protocol)
    loop_item.text = " "
    # delay time
    delay = SubElement(cmd_seq, 'delay')
    delay.text = "5000"
    # copy folder
    copy_dir = SubElement(cmd_seq, 'copy_directory')
    source_dir = SubElement(copy_dir, 'source_path')
    source_dir.text = change_dir.text#cmd_seq.findall('change_directory')[-1].text
    target_dir = SubElement(copy_dir, 'target_path')
    target_dir.text = os.path.join(target_drive, 
                                   os.path.basename(os.path.dirname(source_dir.text)), 
                                   os.path.basename(source_dir.text))
    del_source = SubElement(copy_dir, 'delete_source')
    del_source.text = "True"
    # empty line
    indent(target_dir)
print( prettify(cmd_seq))

<?xml version="1.0" ?>
<command_sequence>
	<!--Hyb 1 for M2-->
	<valve_protocol>Flow TCEP</valve_protocol>
	<valve_protocol>Flow 2xSSC</valve_protocol>
	<valve_protocol>Hybridize 1</valve_protocol>
	<valve_protocol>Incubate 15min</valve_protocol>
	<valve_protocol>Flow Wash Buffer</valve_protocol>
	<valve_protocol>Flow Imaging Buffer</valve_protocol>
	<change_directory>D:\Pu\20220204-P_brain_M1_nonclear\H1M2</change_directory>
	<wakeup>5000</wakeup>
	<loop name="Position Loop Zscan" increment="name">
		<item name="Conv_Zscan_750_647_561_s13_n1000_5Hz"> </item>
	</loop>
	<delay>5000</delay>
	<copy_directory>
		<source_path>D:\Pu\20220204-P_brain_M1_nonclear\H1M2</source_path>
		<target_path>\\KOLMOGOROV\Chromatin_NAS_0\20220204-P_brain_M1_nonclear\H1M2</target_path>
		<delete_source>True</delete_source>
	</copy_directory>
	<!--Hyb 2 for M3-->
	<valve_protocol>Flow TCEP</valve_protocol>
	<valve_protocol>Flow 2xSSC</valve_protocol>
	<valve_protocol>Hybridize 2</valve_protocol>
	<valve_prot

# Depricated

In [4]:
cmd_seq = Element('command_sequence')

for _vid, _hid, _rname in zip(valve_ids, hyb_ids, reg_names):
    # comments
    comment = Comment(f"Hyb {_hid} for {_rname}")
    cmd_seq.append(comment)
    # TCEP
    tcep = SubElement(cmd_seq, 'valve_protocol')
    tcep.text = "Flow TCEP"
    # flow adaptor
    adt = SubElement(cmd_seq, 'valve_protocol')
    adt.text = f"Hybridize {_vid}"
    # delay time
    adt_incubation = SubElement(cmd_seq, 'delay')
    adt_incubation.text = "600000"
    # wash
    wash = SubElement(cmd_seq, 'valve_protocol')
    wash.text = "MERFISH Wash"
    # readouts
    readouts = SubElement(cmd_seq, 'valve_protocol')
    readouts.text = "Flow Imaging Buffer"
    # delay time
    adt_incubation = SubElement(cmd_seq, 'delay')
    adt_incubation.text = "60000"
    # change directory
    change_dir = SubElement(cmd_seq, 'change_directory')
    change_dir.text = os.path.join(source_folder, f"H{_hid}{_rname.upper()}")
    # wakeup
    wakeup = SubElement(cmd_seq, 'wakeup')
    wakeup.text = "5000"
    
    # hybridization loop
    loop = SubElement(cmd_seq, 'loop', name='Position Loop Zscan', increment="name")
    loop_item = SubElement(loop, 'item', name=imaging_protocol)
    loop_item.text = " "
    # delay time
    delay = SubElement(cmd_seq, 'delay')
    delay.text = "5000"
    # copy folder
    copy_dir = SubElement(cmd_seq, 'copy_directory')
    source_dir = SubElement(copy_dir, 'source_path')
    source_dir.text = change_dir.text#cmd_seq.findall('change_directory')[-1].text
    target_dir = SubElement(copy_dir, 'target_path')
    target_dir.text = os.path.join(target_drive, 
                                   os.path.basename(os.path.dirname(source_dir.text)), 
                                   os.path.basename(source_dir.text))
    del_source = SubElement(copy_dir, 'delete_source')
    del_source.text = "True"
    # empty line
    indent(target_dir)
print( prettify(cmd_seq))